## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [1]:
!pip install cassandra-driver
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

In [2]:
import cassandra

#### Creating a Cluster

In [3]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'], port=6000) 
    session = cluster.connect()
except Exception as e:
    print(e)

#### Testing the connection and error handling

In [4]:
try: 
    session.execute("""select * from music_libary""")
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


#### Create & Set Keyspace

In [5]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS nosqldb 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Connecting to keyspace

In [6]:
try:
    session.set_keyspace('nosqldb')
except Exception as e:
    print(e)

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

`Table Name: table1
column 1: Session number
column 2: Item in session number
column 3: Artist name
Column 4: Song title
Cilumn 5: Length
PRIMARY KEY(session_number, item_in_session_number)
PARTITION KEY: session_number
CLUSTERING COLUMN: item_in_session_number`

In [9]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command       
query = "CREATE TABLE IF NOT EXISTS table1 "
query = query + "(session_number int, item_in_session_number int, artist_name varchar, song_title varchar, length float, PRIMARY KEY ((session_number, item_in_session_number)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
import csv
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip the header in the csv file
    for row in csvreader:
## Task: Write the INSERT statements and assign it to the query variable
        #query = "<Place your insert statement and assign the values here>"
        query = "INSERT INTO table1 (session_number, item_in_session_number, artist_name, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s,%s)"
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[8]), int(row[3]), (row[0]), (row[9]), float(row[5])))

### Validate our Data Model using a SELECT

In [20]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT session_number, item_in_session_number, artist_name, song_title, length FROM table1 WHERE session_number = 338 AND item_in_session_number = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Output of Query 1
Faithless Music Matters (Mark Knight Dub) 495.30731201171875

### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

`Table Name: table2
column 1: User Id
column 2: Session number
column 3: Item in session number
column 4: Artist name
Column 5: Song title
Column 6: First name
Column 7: Last name
PRIMARY KEY((user_id, session_number), item_in_session_number))
COMPOSITE PARTITION KEY: user_id, session_number
CLUSTERING COLUMN: item_in_session_number
The clustering column item_in_session_number will determine the sort order within the partition`

In [17]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command     
query = "CREATE TABLE IF NOT EXISTS table2 "
query = query + "(user_id int, session_number int, item_in_session_number int, artist_name varchar, song_title varchar, fname varchar, lname varchar,  \
                    PRIMARY KEY ((user_id, session_number), item_in_session_number))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

### Let's insert our data into of table

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip the header in the csv file
    for row in csvreader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO table2 (user_id, session_number, item_in_session_number, artist_name, song_title, fname, lname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))

### Validate our Data Model using a SELECT

In [21]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT artist_name, song_title, fname, lname FROM table2 WHERE user_id = 10 AND session_number = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.fname, row.lname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Output of Query 2
* Down To The Bone Keep On Keepin' On Sylvie Cruz
* Three Drives Greece 2000 Sylvie Cruz
* Sebastien Tellier Kilometer Sylvie Cruz
* Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz

### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

`Table Name: table3
Column 1: Song title
column 2: User Id
Column 3: First name
Column 4: Last name
PRIMARY KEY((song_title), user_id))
PARTITION KEY: song_title
CLUSTERING COLUMN/KEY: user_id`

In [22]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command  
query = "CREATE TABLE IF NOT EXISTS table3 "
query = query + "(song_title varchar, user_id int, fname varchar, lname varchar, PRIMARY KEY ((song_title), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [23]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip the header in the csv file
    for row in csvreader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO table3 (song_title, user_id, fname, lname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[9], int(row[10]), row[1], row[4]))

### Validate our Data Model using a SELECT

In [25]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT fname, lname FROM table3 WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.fname, row.lname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Output of Query 3
* Jacqueline Lynch
* Tegan Levine
* Sara Johnson

### Drop the tables before closing out the sessions

In [28]:
query = "drop table table1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [29]:
query = "drop table table2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [30]:
query = "drop table table3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()